In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine("sqlite:///./app/connection/test.db", echo=False)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
session = SessionLocal()

In [2]:
import numpy as np
from app.services.kor_fs_service import KorFsService
from app.services.kor_value_service import KorValueService


kor_fs_service = KorFsService(session)
kor_value_service = KorValueService(session)

fs_list = kor_fs_service.get_fs("y", ["매출총이익", "자산"])
value_list = kor_value_service.get_values_by_metrics("PBR", isDf=True)

In [8]:
# 벨류지표
value_list.loc[value_list["amt"] < 0, "amt"] = np.nan
value_pivot = value_list.pivot(index="itemCd", columns="metrics", values="amt")

# 퀄리티 지표
fs_list = fs_list.sort_values(["itemCd", "account", "baseDt"])
fs_list = fs_list.groupby(["itemCd", "account"]).tail(1)
fs_list_pivot = fs_list.pivot(index="itemCd", columns="account", values="amt")
fs_list_pivot["GPA"] = fs_list_pivot["매출총이익"] / fs_list_pivot["자산"]

# 데이터 합치기
bind_rank = (
    value_pivot["PBR"]
    .rank()
    .to_frame()
    .merge(
        fs_list_pivot["GPA"].rank(ascending=False),
        how="inner",
        on="itemCd",
    )
)

# 상관관계
bind_rank.corr()
"""
음의 상관관계
	PBR	        GPA
PBR	1.000000	-0.087968
GPA	-0.087968	1.000000
"""

In [9]:
# PBR의 분위수 별 GPA의 평균 계산

,PBR,GPA
PBR,1.000000,-0.087968
GPA,-0.087968,1.000000
